In [134]:
import torch
torch.manual_seed(0)
import numpy as np
from numpy.lib.stride_tricks import as_strided

#initiate the input as Batch_size, Channel, height, width 1*3*32*32
inputs = torch.randn(1,3, 32, 32)
inputs.shape
input_np = inputs.numpy()
input_np.shape

(1, 3, 32, 32)

#  Max Pooling

In [135]:
MaxPool=torch.nn.MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, return_indices=False, ceil_mode=False)
torch_out=MaxPool(inputs)
torch_out = torch_out.numpy()
torch_out.shape

(1, 3, 31, 31)

 
    2D Pooling
    Parameters:
        A: input 2D array
        kernel_size: int, the size of the window
        stride: int, the stride of the window
        padding: int, implicit zero paddings on both sides of the input
        pool_mode: string, 'max' or 'avg'
    

In [136]:
def pool2d(img, kernel_size, stride, padding, pool_mode='max'):
    
    my_out = np.empty((1,3,(img.shape[2] - kernel_size)//stride + 1,
                    (img.shape[3] - kernel_size)//stride + 1), dtype = float)
    
    for batch in img:
        i = 0
        for A in batch:
            A = np.pad(A, padding, mode='constant')
            
            output_shape = ((A.shape[0] - kernel_size)//stride + 1,
                    (A.shape[1] - kernel_size)//stride + 1)
            kernel_sizes = (kernel_size, kernel_size)
            A_w = as_strided(A, shape = output_shape + kernel_sizes, 
                        strides = (stride*A.strides[0],
                                   stride*A.strides[1]) + A.strides)
            A_w = A_w.reshape(-1, *kernel_sizes)


            if pool_mode == 'max':
                my_out[0,i] = A_w.max(axis=(1,2)).reshape(output_shape)
            elif pool_mode == 'avg':
                my_out[0,i] =  A_w.mean(axis=(1,2)).reshape(output_shape)
            i+=1
    return my_out

my_out = pool2d(input_np, kernel_size=2, stride=1, padding=0, pool_mode='max')
my_out.shape

(1, 3, 31, 31)

In [81]:
torch_out == my_out

array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  T

#  Average pooling

In [146]:
AvgPool=torch.nn.AvgPool2d(kernel_size=2, stride=1, padding=0,ceil_mode=False, count_include_pad=True,divisor_override=None)
torch_out=AvgPool(inputs)
torch_out = torch_out.numpy()
torch_out.shape

(1, 3, 31, 31)

In [83]:
def pool2d(A, kernel_size, stride, padding, pool_mode='max'):
    
    A = np.pad(A, padding, mode='constant')

    output_shape = ((A.shape[0] - kernel_size)//stride + 1,
                    (A.shape[1] - kernel_size)//stride + 1)
    kernel_size = (kernel_size, kernel_size)
    A_w = as_strided(A, shape = output_shape + kernel_size, 
                        strides = (stride*A.strides[0],
                                   stride*A.strides[1]) + A.strides)
    A_w = A_w.reshape(-1, *kernel_size)

    if pool_mode == 'max':
        return A_w.max(axis=(1,2)).reshape(output_shape)
    elif pool_mode == 'avg':
        return A_w.mean(axis=(1,2)).reshape(output_shape)

my_out = np.empty((1,3,31,31), dtype = float)
for batch in input_np:
    i = 0
    for channel in batch:
        my_out[0,i] =  pool2d(channel, kernel_size=2, stride=1, padding=0, pool_mode='avg')
        i+=1

In [84]:
torch_out == my_out

array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  T

# Conv NN

In [85]:
torch.manual_seed(0)
Conv=torch.nn.Conv2d(in_channels=3, out_channels=6,kernel_size=3, stride=1, padding=0, dilation=1, groups=1,
                     bias=True, padding_mode='zeros')
torch_out=Conv(inputs)
torch_out.shape

torch.Size([1, 6, 30, 30])

In [86]:
weight = Conv.weight.data
bias = Conv.bias.data
def conv2d(inputs, weight, bias, stride):
    padding = 0
    N, C, H, W = inputs.shape
    filter_, _, kernel_H, kernel_W = weight.shape
    
    H_out= 1+(H+2*padding-kernel_H)//stride
    W_out= 1+(W+2*padding-kernel_W)//stride
    out = torch.zeros(N, filter_, H_out, W_out)
    
    for batch in range(N):
        for filter_i in range(filter_):
            for row in range(H_out):
                for col in range(W_out):
                    out[batch, filter_i, row, col] = torch.sum(inputs[batch, :, row*stride:row*stride + kernel_H, 
                                                                     col*stride:col*stride + kernel_W] *weight[filter_i, :,:,:]) + bias[filter_i]
                    
    return out

my_out = conv2d(inputs, weight, bias, 1)
my_out.shape

torch.Size([1, 6, 30, 30])

In [87]:
torch_out - my_out<=0.000001

tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ...

#  Conv 2D with Dilation

In [88]:
Conv=torch.nn.Conv2d(in_channels=3, out_channels=6,kernel_size=5, stride=2, padding=0, dilation=2, groups=1,
                     bias=True, padding_mode='zeros')
torch_out=Conv(inputs)
torch_out

tensor([[[[-6.7205e-01,  6.9666e-01, -5.0877e-01, -3.7890e-02, -2.4523e-01,
            8.1609e-02,  4.2619e-01, -9.0766e-01,  3.3815e-01, -5.6090e-01,
           -1.0934e-01,  3.2475e-02],
          [ 6.2436e-01,  5.6893e-02, -6.0532e-01, -3.9386e-01,  1.5883e-01,
            3.4871e-01,  5.1953e-01,  2.8136e-01, -8.8076e-02,  3.3201e-01,
           -5.2483e-01, -4.8972e-01],
          [ 3.2871e-01,  7.7371e-01,  6.6600e-01, -1.1036e+00,  1.8207e-01,
            6.3458e-01, -1.8130e-01,  6.8301e-01, -9.6772e-02,  1.1915e+00,
           -8.5092e-01, -7.4201e-02],
          [ 1.4339e-01, -3.6058e-02, -9.2370e-02,  7.9441e-01,  7.6642e-01,
            9.4264e-01, -6.5509e-01, -2.0474e-01,  1.2391e+00, -8.2072e-01,
            9.3929e-01,  1.8113e-01],
          [ 4.3519e-01,  1.0018e+00, -1.0260e+00,  7.6188e-01,  7.7489e-02,
            8.3154e-01, -1.0570e+00, -1.4593e-01, -6.2887e-01, -6.4637e-02,
            2.4616e-01,  5.9856e-01],
          [-6.4982e-01,  1.0849e-02,  6.0189e-01, 

In [89]:
kernels = Conv.weight.data
bias = Conv.bias.data

def one_kernel_dilation(kernel, dilation):
    dilation_kernel = torch.zeros([len(kernel)+(dilation-1)*(len(kernel)-1), len(kernel)+(dilation-1)*(len(kernel)-1)])
    count = 0
    for i in range(0, len(dilation_kernel), dilation):
        for j in range(0, len(dilation_kernel), dilation):
            dilation_kernel[i][j] = kernel.flatten()[count]
            count+=1
    return dilation_kernel

def all_kernel_dilation(kernel, dilation):
    kernels = torch.zeros([len(kernel), len(kernel[0]), len(kernel[0][0])+(dilation-1)*(len(kernel[0][0])-1), len(kernel[0][0])+(dilation-1)*(len(kernel[0][0])-1)])
    for i in range(len(kernel)):
        for j in range(len(kernel[0])):
            kernels[i][j] = one_kernel_dilation(kernel[i][j], dilation)
    return kernels

def convolution2D(inputs, kernel, bias, stride, dilation):
    if dilation != 1:
        kernels = all_kernel_dilation(kernel, dilation=dilation)
    out = conv2d(inputs, kernels, bias, stride)
    return out

# test multi channel and multi kernel

my_out = convolution2D(inputs, kernels, bias, 2, 2)
my_out

tensor([[[[-6.7205e-01,  6.9666e-01, -5.0877e-01, -3.7890e-02, -2.4523e-01,
            8.1609e-02,  4.2619e-01, -9.0766e-01,  3.3815e-01, -5.6090e-01,
           -1.0934e-01,  3.2475e-02],
          [ 6.2436e-01,  5.6893e-02, -6.0532e-01, -3.9386e-01,  1.5883e-01,
            3.4871e-01,  5.1953e-01,  2.8136e-01, -8.8076e-02,  3.3201e-01,
           -5.2483e-01, -4.8972e-01],
          [ 3.2871e-01,  7.7371e-01,  6.6600e-01, -1.1036e+00,  1.8207e-01,
            6.3458e-01, -1.8130e-01,  6.8301e-01, -9.6772e-02,  1.1915e+00,
           -8.5092e-01, -7.4201e-02],
          [ 1.4339e-01, -3.6058e-02, -9.2370e-02,  7.9441e-01,  7.6643e-01,
            9.4264e-01, -6.5509e-01, -2.0474e-01,  1.2391e+00, -8.2072e-01,
            9.3929e-01,  1.8113e-01],
          [ 4.3519e-01,  1.0018e+00, -1.0260e+00,  7.6188e-01,  7.7489e-02,
            8.3154e-01, -1.0570e+00, -1.4593e-01, -6.2887e-01, -6.4637e-02,
            2.4616e-01,  5.9856e-01],
          [-6.4982e-01,  1.0849e-02,  6.0189e-01, 

In [90]:
torch_out - my_out<=0.000001

tensor([[[[True, True, True, True, True, True, True, True, True, True, True,
           True],
          [True, True, True, True, True, True, True, True, True, True, True,
           True],
          [True, True, True, True, True, True, True, True, True, True, True,
           True],
          [True, True, True, True, True, True, True, True, True, True, True,
           True],
          [True, True, True, True, True, True, True, True, True, True, True,
           True],
          [True, True, True, True, True, True, True, True, True, True, True,
           True],
          [True, True, True, True, True, True, True, True, True, True, True,
           True],
          [True, True, True, True, True, True, True, True, True, True, True,
           True],
          [True, True, True, True, True, True, True, True, True, True, True,
           True],
          [True, True, True, True, True, True, True, True, True, True, True,
           True],
          [True, True, True, True, True, True, Tru

# Conv2D Transpose

In [91]:
ConvTranspose = torch.nn.ConvTranspose2d(in_channels=3, out_channels=4,kernel_size=3, stride=1, padding=0, output_padding=0,
groups=1, bias=True, dilation=1, padding_mode='zeros')
torch_out=ConvTranspose(inputs)
torch_out

tensor([[[[ 1.3891e-01,  2.1060e-01, -1.8841e-01,  ...,  3.0564e-01,
            1.4281e-02, -3.8213e-02],
          [ 2.2454e-01,  1.4816e-01,  4.4866e-02,  ..., -4.9636e-01,
           -4.8723e-02,  1.3326e-01],
          [-2.0076e-01, -3.8757e-01, -2.1847e-01,  ..., -6.6750e-01,
           -4.0777e-01, -1.5830e-02],
          ...,
          [-1.9467e-01,  8.4558e-01, -2.4026e-01,  ...,  9.3834e-02,
           -2.2740e-01, -1.4756e-01],
          [-3.8045e-01,  7.6494e-01, -9.1256e-01,  ..., -7.7247e-01,
            1.0374e-01, -1.1644e-02],
          [-1.4653e-01, -1.7017e-01, -2.7070e-01,  ..., -2.1440e-01,
            2.6342e-01, -9.4709e-02]],

         [[-1.5969e-01,  8.3216e-02,  4.5905e-02,  ...,  2.2385e-02,
           -2.4490e-01,  2.9391e-01],
          [-1.9731e-02,  3.6226e-01, -7.7498e-04,  ..., -1.9056e-01,
            1.2951e-01,  2.3212e-01],
          [-3.5382e-01,  1.9782e-01, -4.3353e-01,  ..., -6.0119e-01,
           -1.2725e-01, -4.8110e-02],
          ...,
     

In [92]:
weight = ConvTranspose.weight.data
bias = ConvTranspose.bias.data
def transpose_conv(inputs, weight, bias):
    in_, out_, H, W = weight.shape
    out = torch.zeros(1, out_, inputs.shape[2]+H-1, inputs.shape[3]+W-1)
    for batch in range(1):
        for out_channel in range(out_):
            for in_channel in range(in_):
                for row in range(inputs.shape[2]):
                    for col in range(inputs.shape[3]):
                        out[batch, out_channel, row:row+H, col:col+W] += inputs[batch, in_channel, row, col] * weight[in_channel,out_channel,:,:]
            out[batch,out_channel,:,:] +=bias[out_channel]
    return out                                             
my_out = transpose_conv(inputs, weight, bias)
my_out

tensor([[[[ 1.3891e-01,  2.1060e-01, -1.8841e-01,  ...,  3.0564e-01,
            1.4281e-02, -3.8213e-02],
          [ 2.2454e-01,  1.4816e-01,  4.4866e-02,  ..., -4.9636e-01,
           -4.8723e-02,  1.3326e-01],
          [-2.0076e-01, -3.8757e-01, -2.1847e-01,  ..., -6.6750e-01,
           -4.0777e-01, -1.5830e-02],
          ...,
          [-1.9467e-01,  8.4558e-01, -2.4026e-01,  ...,  9.3834e-02,
           -2.2740e-01, -1.4756e-01],
          [-3.8045e-01,  7.6494e-01, -9.1256e-01,  ..., -7.7247e-01,
            1.0374e-01, -1.1644e-02],
          [-1.4653e-01, -1.7017e-01, -2.7070e-01,  ..., -2.1440e-01,
            2.6342e-01, -9.4709e-02]],

         [[-1.5969e-01,  8.3216e-02,  4.5905e-02,  ...,  2.2385e-02,
           -2.4490e-01,  2.9391e-01],
          [-1.9731e-02,  3.6226e-01, -7.7501e-04,  ..., -1.9056e-01,
            1.2951e-01,  2.3212e-01],
          [-3.5382e-01,  1.9782e-01, -4.3353e-01,  ..., -6.0119e-01,
           -1.2725e-01, -4.8110e-02],
          ...,
     

In [93]:
torch_out - my_out<=0.000001

tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ...

#  Flatten

In [94]:
torch_out = torch.flatten(inputs, start_dim=0, end_dim=-1)
torch_out = torch_out.numpy()
torch_out

array([-1.1258398 , -1.1523602 , -0.25057858, ..., -0.6380098 ,
       -1.1713753 , -0.8415489 ], dtype=float32)

In [95]:
my_out = input_np.flatten()
my_out

array([-1.1258398 , -1.1523602 , -0.25057858, ..., -0.6380098 ,
       -1.1713753 , -0.8415489 ], dtype=float32)

In [96]:
torch_out == my_out

array([ True,  True,  True, ...,  True,  True,  True])

#  Sigmoid

In [97]:
torch_out = torch.sigmoid(inputs,  out=None)
torch_out = torch_out.numpy()
torch_out = np.around(torch_out, decimals=4, out=None)
torch_out 

array([[[[0.2449, 0.2401, 0.4377, ..., 0.8301, 0.7204, 0.3008],
         [0.3512, 0.5079, 0.3793, ..., 0.2255, 0.8605, 0.3655],
         [0.3614, 0.715 , 0.7523, ..., 0.2776, 0.5084, 0.6704],
         ...,
         [0.7343, 0.7836, 0.8891, ..., 0.6934, 0.361 , 0.4702],
         [0.3948, 0.693 , 0.1959, ..., 0.4652, 0.4091, 0.3876],
         [0.2151, 0.6685, 0.129 , ..., 0.6312, 0.747 , 0.1531]],

        [[0.2527, 0.4679, 0.336 , ..., 0.2724, 0.6723, 0.6083],
         [0.3631, 0.6601, 0.1088, ..., 0.3139, 0.1052, 0.5827],
         [0.2455, 0.6504, 0.2939, ..., 0.7142, 0.5742, 0.5442],
         ...,
         [0.9068, 0.2866, 0.6249, ..., 0.6195, 0.2836, 0.3601],
         [0.6748, 0.2832, 0.5435, ..., 0.3302, 0.2929, 0.3567],
         [0.3023, 0.1376, 0.6169, ..., 0.4058, 0.1586, 0.7478]],

        [[0.6605, 0.7078, 0.6147, ..., 0.6837, 0.3012, 0.8577],
         [0.5479, 0.4557, 0.5797, ..., 0.2384, 0.3562, 0.6782],
         [0.2338, 0.3053, 0.6468, ..., 0.4525, 0.5625, 0.7948],
        

In [98]:
def sigmoid(x):
        return 1.0/(1+np.exp(-x))
my_out = input_np
my_out = sigmoid(my_out)
my_out = np.around(my_out, decimals=4, out=None)
my_out

array([[[[0.2449, 0.2401, 0.4377, ..., 0.8301, 0.7204, 0.3008],
         [0.3512, 0.5079, 0.3793, ..., 0.2255, 0.8605, 0.3655],
         [0.3614, 0.715 , 0.7523, ..., 0.2776, 0.5084, 0.6704],
         ...,
         [0.7343, 0.7836, 0.8891, ..., 0.6934, 0.361 , 0.4702],
         [0.3948, 0.693 , 0.1959, ..., 0.4652, 0.4091, 0.3876],
         [0.2151, 0.6685, 0.129 , ..., 0.6312, 0.747 , 0.1531]],

        [[0.2527, 0.4679, 0.336 , ..., 0.2724, 0.6723, 0.6083],
         [0.3631, 0.6601, 0.1088, ..., 0.3139, 0.1052, 0.5827],
         [0.2455, 0.6504, 0.2939, ..., 0.7142, 0.5742, 0.5442],
         ...,
         [0.9068, 0.2866, 0.6249, ..., 0.6195, 0.2836, 0.3601],
         [0.6748, 0.2832, 0.5435, ..., 0.3302, 0.2929, 0.3567],
         [0.3023, 0.1376, 0.6169, ..., 0.4058, 0.1586, 0.7478]],

        [[0.6605, 0.7078, 0.6147, ..., 0.6837, 0.3012, 0.8577],
         [0.5479, 0.4557, 0.5797, ..., 0.2384, 0.3562, 0.6782],
         [0.2338, 0.3053, 0.6468, ..., 0.4525, 0.5625, 0.7948],
        

In [99]:
torch_out == my_out

array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  T

#  ROI pooling

In [128]:
rois_in_feature = torch.Tensor([[0, 0, 0, 3, 3]])
import torchvision
torch_out =torchvision.ops.roi_pool(input = inputs,  boxes=rois_in_feature, output_size = 1,spatial_scale = 1.0)
torch_out = torch_out.numpy()
torch_out

array([[[[1.645867 ]],

        [[0.8777172]],

        [[1.8793321]]]], dtype=float32)


    param feature_map: (1, C, H, W)
    param rois: (1, N, 4) N refers to bbox num, 4 represent (ltx, lty, w, h) 
    param size: output size
    return: (1, C, size[0], size[1])


In [125]:
def roi_pooling(feature_map, rois, size=(1, 1)):
    
    output = []
    rois_num = rois.size(1)

    for i in range(rois_num):
        roi = rois[0][i]
        x, y, w, h = roi
        
        output.append(pool2d(feature_map[:, :, y:y+h+1, x:x+w+1], 4,1,0, pool_mode='max'))
    return output

if __name__ == '__main__':
    test_tensor = inputs
    #test_tensor = test_tensor.view(1, 1, 8, 8)
    rois = torch.tensor([[0, 0, 3, 3]])
    rois = rois.view(1, -1, 4)
    
    my_out = roi_pooling(input_np, rois, (1, 1))
    print(my_out )

[array([[[[1.64586699]],

        [[0.8777172 ]],

        [[1.87933207]]]])]


In [130]:
torch_out == my_out

array([[[[[ True]],

         [[ True]],

         [[ True]]]]])

#  Batch norm

In [102]:
mean = inputs.view(3,-1)
running_mean = torch.mean(mean, 1)
var = inputs.view(3,-1)
running_var = torch.var(var, 1)
torch_out =torch.nn.functional.batch_norm(inputs, running_mean, running_var,weight=None, bias=None, training=False, momentum=0.1,eps=1e-05)
torch_out.shape

torch.Size([1, 3, 32, 32])

In [103]:
def batchnorm_forward(x,  running_mean, running_var, mode , momentum,eps ):
    N, C, H, W = x.shape
    out= None
    if mode == 'train':
        sample_mean = running_mean
        sample_var = running_var

        # Normalization followed by Affine transformation
        x_normalized = (x[0,:,:,:] - sample_mean.reshape(( 1,C, 1, 1)))/np.sqrt(sample_var.reshape(( 1,C, 1, 1)) + eps)
        out = x_normalized

        # Estimate running average of mean and variance to use at test time
        running_mean = momentum * running_mean + (1 - momentum) * sample_mean
        running_var = momentum * running_var + (1 - momentum) * sample_var
        
    elif mode == 'test':
        # normalize using running average
        running_mean = momentum * running_mean + (1 - momentum) * running_mean
        running_var = momentum * running_var + (1 - momentum) * running_var
        x_normalized = (x[0,:,:,:] - running_mean.reshape(( 1,C, 1, 1)))/np.sqrt(running_var.reshape((1, C, 1, 1)) + eps)
        # Learned affine transformation
        out = x_normalized 
    return out
running_mean = running_mean.numpy()
running_var = running_var.numpy()
my_out = batchnorm_forward(input_np,  running_mean, running_var,  'test', 0.1,1e-05 )
my_out.shape

(1, 3, 32, 32)

In [104]:
torch_out - my_out<=0.000001

tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ...

# Cross Entropy

In [140]:
target = torch.randint(3, (1,32,32), dtype=torch.int64)
torch_out = torch.nn.functional.cross_entropy(inputs, target, weight=None,
size_average=None, ignore_index=-100, reduce=None,reduction='mean')
torch_out                                                          

tensor(1.3668)

In [144]:
def compute_loss(softmax, target):
    loss_list = torch.zeros([len(inputs)])
    for k in range(len(softmax)):
        loss = 0
        for i in range(target[k].shape[1]):
            for j in range(target.shape[2]):
                index = target[k, i, j]
                num = softmax[k, index, i ,j]
                loss -= num
        loss_list[k] = -loss / target.shape[1] / target.shape[2]
    return loss_list.mean()

def cross_entropy(x, target):
    #compute softmax
    x_log_softmax = torch.zeros(x.shape)
    for i in range(len(x)):
        exp = torch.exp(x[i])
        sum = torch.sum(exp, dim=0)
        softmax = exp/sum
        x_log_softmax[i] = -torch.log(softmax)

    ce = compute_loss(x_log_softmax, target)
    return ce
my_out = cross_entropy(inputs, target)
my_out

tensor(1.3668)

In [145]:
torch_out == my_out

tensor(True)

# MSE Loss


Input: (N, *)(N,∗) where *∗ means, any number of additional dimensions

Target: (N, *)(N,∗) , same shape as the input

In [108]:
target = torch.randn(1,3, 32, 32)
torch_out =torch.nn.functional.mse_loss(inputs, target, size_average=None,reduce=None, reduction='mean')
torch_out  

tensor(1.9392)

In [109]:
def mse_loss(input, target):
    square = (input - target)**2
    return square.mean()
my_out = mse_loss(inputs, target)
print(my_out)

tensor(1.9392)


In [110]:
torch_out == my_out

tensor(True)